## Load settings from .env file

In [ ]:
!python -m pip install python-dotenv==1.0.0
import os
from dotenv import load_dotenv
if not load_dotenv(): raise Exception(".env file not found")

## Prepare kernel using Azure Cognitive Search

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.connectors.memory.azure_search import AzureSearchMemoryStore

kernel = Kernel()

kernel.add_text_embedding_generation_service(
    "ada",
    AzureTextEmbedding(
        os.getenv("AZURE_OPENAI_EMBEDDING_MODEL"),
        os.getenv("AZURE_OPENAI_ENDPOINT"),
        os.getenv("AZURE_OPENAI_API_KEY"),
    ),
)

kernel.add_chat_service(
    "chat",
    AzureChatCompletion(
        os.getenv("AZURE_OPENAI_CHAT_MODEL"),
        os.getenv("AZURE_OPENAI_ENDPOINT"),
        os.getenv("AZURE_OPENAI_API_KEY"),
    ),
)

kernel.register_memory_store(
    memory_store=AzureSearchMemoryStore(
        1536, os.getenv("AZURE_SEARCH_ENDPOINT"), os.getenv("AZURE_SEARCH_API_KEY")
    )
)

## Grounding prompts (semantic functions) with Memory "recall"

In [ ]:
from semantic_kernel.core_skills import TextMemorySkill

kernel.import_skill(TextMemorySkill())

sk_prompt = """
Considering these facts

About me: {{recall 'where did I grow up?'}}
About me: {{recall 'where do I live?'}}

Question: {{$input}}

Provide a concise answer ('Answer: ') and a separate explanation ('Explanation: '), in two lines.
"""

sk_function = kernel.create_semantic_function(prompt_template=sk_prompt, max_tokens=200)

In [ ]:
context= kernel.create_new_context()
context[TextMemorySkill.COLLECTION_PARAM] = "aboutMeUser002";

result = await sk_function.invoke_async(input="Do I live in the same town where I grew up?", context=context)

print(result)